In [113]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import colors
import threading, queue
import scipy.io


from Tree import Node


In [114]:
#split s means we check all y-values to see if they're above or below the line
# %%
#split s means we check all y-values to see if they're above or below the line
'''
split cost functions
Desc: finds cost of a split
parameters: 
s: split value
c1: mean color left/below split [r,g,b]
c2: mean color right/above split [r,g,b]
'''
def hor_split_cost(s, y_values, vals):
    #get values left
    left_values = vals[np.nonzero(y_values[y_values <= s])]
    right_values = vals[np.nonzero(y_values[y_values > s])]
    print(np.shape(left_values))
    c1 = np.mean(left_values, axis = 0)
    c2 = np.mean(right_values, axis = 0)
    #print(len(left_values), len(right_values))
    #print(np.shape(np.tile(c1, (np.size(left_values), 1))))
    #print(left_values, np.tile(c1, (np.size(left_values), 1)))
    left_values -= np.tile(c1, (np.size(left_values), 1))
    left_cost = np.sum(np.square(np.linalg.norm(left_values, axis=1)))
        
    right_values -= np.tile(c2, (np.size(right_values), 1))
    right_cost = np.sum(np.square(np.linalg.norm(right_values, axis=1)))
    cost = right_cost + left_cost
    return cost, c1, c2

def vert_split_cost(s, x_values, vals):
    #get values left 
    left_values = vals[np.nonzero(x_values <= s)]
    right_values = vals[np.nonzero(x_values > s)]
    c1 = np.mean(left_values, axis = 0)
    c2 = np.mean(right_values, axis = 0)
    #print(len(left_values))
    #print(c1)
    left_values -= np.tile(c1, (np.size(left_values), 1))
    left_cost = np.sum(np.square(np.linalg.norm(left_values, axis=1)))
        
    right_values -= np.tile(c2, (len(right_values), 1))
    right_cost = np.sum(np.square(np.linalg.norm(right_values, axis=1)))

    #print(left_values)
    cost = left_cost + right_cost
    return cost, c1, c2

In [115]:
'''
Returns direction & optimal split value
'''
def OptimalSplitRegression(x_data,y_data,vals):
    #print(x_data,y_data,vals)
    #pre-sort X & Y values:
    x_sorted = np.sort(x_data)
    y_sorted = np.sort(y_data)
    if(len(x_sorted) == 0 and len(y_sorted) == 0):
        return False
    #print(x_sorted)

    #get split values by summing array with shifted array by 1 to right and taking mean of every entry
    x_splits = 0.5 * (x_sorted[:-1]+np.roll(x_sorted,-1)[:-1])
    y_splits = 0.5 * (y_sorted[:-1]+np.roll(y_sorted,-1)[:-1])
    #print(x_splits, y_splits)
     
    min_cost = float('inf')
    min_c1: np.array
    min_c2: np.array
    min_direction = False
    min_s = 0

    #find min cost split among both x and y splits
    for x_split in x_splits:
        cost, c1, c2 = hor_split_cost(x_split, y_data, vals)
        print(f'x_split: {x_split}; cost: {cost}')
        if(cost < min_cost):
            min_s = x_split
            min_cost = cost
            min_c1 = c1
            min_c2 = c2
            min_direction = True
    for y_split in y_splits:
        cost, c1, c2 = vert_split_cost(y_split, x_data, vals)
        print(f'y_split: {y_split}; cost: {cost}')
        if(cost <= min_cost):
            min_s = y_split
            min_cost = cost
            min_c1 = c1
            min_c2 = c2
            min_direction = False
    return min_s, min_cost, min_c1, min_c2, min_direction

In [116]:
def write_to_image(img, dir, x_min, x_max, y_min, y_max, s, c1, c2, x_dim, y_dim):
    #print(dir, x_min, x_max, y_min, y_max, s, c1, c2, x_dim, y_dim)
    if(dir):
        #horizontal cut

        #scale it to the integers
        #split = s*x_dim
        img[x_min:x_max, y_min:int(s)] = c1
        img[x_min:x_max, int(s):y_max] = c2
    else:
        #vertical cut

        #scale it to the integers
        #split  = s* y_dim
        img[x_min:int(s), y_min:y_max] = c1
        img[int(s):x_max, y_min:y_max] = c2

    return img

In [121]:
#get matlab data as nested dictionary
image = scipy.io.loadmat('MysteryImage.mat')

#extract compressed matrix from dictionaries

#columns: y-axis; rows: x-axis; vals: [r,g,b]
Y = np.array(image['cols'])
X = np.array(image['rows'])
V = np.array(image['vals'])

print(V)
print(V[np.nonzero(Y[Y>3])])

# number of pixels
n = np.shape(X)[0]

[[0.25098039 0.14509804 0.11764706]
 [0.97254902 0.96470588 0.96862745]
 [0.92941176 0.94117647 0.96078431]
 ...
 [0.58431373 0.58431373 0.55294118]
 [0.62745098 0.63529412 0.63137255]
 [0.67058824 0.65882353 0.63921569]]
[[0.25098039 0.14509804 0.11764706]
 [0.97254902 0.96470588 0.96862745]
 [0.92941176 0.94117647 0.96078431]
 ...
 [0.68235294 0.67843137 0.65882353]
 [0.63529412 0.65098039 0.65490196]
 [0.0627451  0.03529412 0.00392157]]


In [118]:
#Code for Regression Tree:
MAX_ITTER = 2
x_dim = 2592
y_dim = 1456

n = 10

#stores leaves to consider
q = queue.Queue()
# X_points, Y_points, x_min, x_max, y_min, y_max, color
init_node = Node(X, Y, V, 0, x_dim, 0, y_dim)

q.put(init_node)
itter = 0
img = np.zeros((x_dim, y_dim, 3))

#q.qsize() function is inaccurate, so we use our own count
queue_count = 1
while itter < MAX_ITTER and queue_count > 0:
    itter += 1
    curr_node = q.get()
    queue_count -= 1
    #print(curr_node.vals)
    s, cost, c1, c2, direction = OptimalSplitRegression(curr_node.x_elements,curr_node.y_elements,curr_node.vals)
    print(curr_node.x_elements, curr_node.y_elements)
    #write to our image with the current split
    #print(f"c1: {c1}, c2: {c2}")
    write_to_image(img, direction, curr_node.x_min, curr_node.x_max, curr_node.y_min, curr_node.y_max, s, c1, c2, x_dim, y_dim)
    if(direction):
        y_left = curr_node.y_elements[curr_node.y_elements <= s]
        y_right = curr_node.y_elements[curr_node.y_elements > s]

        lefts = np.nonzero(curr_node.y_elements <= s)
        rights = np.nonzero(curr_node.y_elements > s)

        left_values = curr_node.vals[lefts]
        right_values = curr_node.vals[rights]

        #create children node we push to the queue: 
        left_node = Node(curr_node.x_elements[lefts], y_left, left_values, curr_node.x_min, curr_node.x_max, curr_node.y_min, int(s))
        right_node = Node(curr_node.x_elements[rights], y_right, right_values, curr_node.x_min, curr_node.x_max, int(s), curr_node.y_max)
        #print("len lengths: ", len(lefts), len(rights))

        #enqueue mixed leaves:
        if(len(lefts) > 1 and cost > 0):
            q.put(left_node)
            queue_count += 1
        if(len(rights) > 1 and cost > 0):
            q.put(right_node)
            queue_count += 1
    else:
        x_left = curr_node.x_elements[curr_node.x_elements <= s]
        x_right = curr_node.x_elements[curr_node.x_elements > s]
        lefts = np.nonzero(curr_node.x_elements <= s)
        rights = np.nonzero(curr_node.x_elements > s)
        left_values = curr_node.vals[lefts]
        right_values = curr_node.vals[rights]
        
        #create children node we push to the queue:
        left_node = Node(x_left, curr_node.y_elements[lefts], left_values, curr_node.x_min, int(s), curr_node.y_min, curr_node.y_max)
        right_node = Node(x_right, curr_node.y_elements[rights], right_values, int(s), curr_node.x_max, curr_node.y_min, curr_node.y_max)
        print("len lengths: ", lefts, rights, np.size(rights))

        if(np.size(lefts) > 1 and cost > 0):
            q.put(left_node)
            queue_count += 1
        if(np.size(rights) > 1 and cost > 0):
            q.put(right_node)
            queue_count +=1
#print(itter, queue_count)
plt.imshow(img)
'''
we need top get the indices of the values and then 
'''

(6607,)


ValueError: non-broadcastable output operand with shape (6607,) doesn't match the broadcast shape (6607,6607)